## Mount adls containers for the project


In [0]:
def mount_adls(storage_account_name, container_name):
    # get secrets from Azure key vault
    clientID = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-clientid')
    tenantId = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-tenantId')
    client_secret = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-client-secret')

    # set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": clientID,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenantId}/oauth2/token"}
    
    # unmount the mount point if already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
          dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    #  mount starage account containers
    dbutils.fs.mount(
                    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                    mount_point = f"/mnt/{storage_account_name}/{container_name}",
                    extra_configs = configs)
    
    display(dbutils.fs.mounts())


In [0]:
mount_adls('dldatabrickscoursedev001','raw')

In [0]:
mount_adls('dldatabrickscoursedev001','demo')

In [0]:
mount_adls('dldatabrickscoursedev001','presentation')

In [0]:
mount_adls('dldatabrickscoursedev001','processed')